In [6]:
# Import plotting libraries
import matplotlib 
from matplotlib import pyplot as plt
matplotlib.rcParams.update({'font.size': 20})
plt.rcParams.update({'font.size': 22})

import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

print(keras.__version__)
import numpy as np

print(np.__version__)
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow

%load_ext autoreload
%autoreload 2


2.3.1
1.21.2
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
seed = 12

In [8]:
data = pd.read_csv('bg_struct.csv')

In [9]:
data.head()

,Unnamed: 0,pretty_formula,bandgap,cell_length_a,cell_lentgh_b,cell_lentgh_c,cell_angle_alpha,cell_angle_beta,cell_angle_gamma
0,0,BrCl,1.8278,4.645947,4.645947,7.810701,90.000000,90.000000,115.948189
1,1,SrC2,1.8008,3.710786,3.710786,3.710785,85.434482,85.434482,85.434493
2,2,Cs2Se,1.8027,5.624158,9.095804,10.998913,90.000000,90.000000,90.000000
3,3,Cs2Se,1.8141,9.136351,5.630188,12.173633,63.874547,90.000000,90.000000
4,4,CsPbBr3,1.8178,8.459612,8.459612,8.459612,120.097716,120.097716,89.830834


In [10]:
data.shape

(16335, 9)

In [11]:
X = data[['cell_length_a', 'cell_lentgh_b', 'cell_lentgh_c', 'cell_angle_alpha', 'cell_angle_beta', 'cell_angle_gamma']].values
Y = data[['bandgap']].values

In [12]:
# create test/train split from the data above and  keep 20% for testing
X_train_pn, X_test_pn, y_train, y_test = train_test_split(X, Y,
                                                         test_size=0.20, 
                                                         random_state=seed)

# train a standard scaler model to apply it to the train set and test set
X_train_scaler = StandardScaler().fit(X_train_pn)
X_train = X_train_scaler.transform(X_train_pn)
X_test = X_train_scaler.transform(X_test_pn)


### Create a neural network model

In [30]:
def neural_network_model(n1, n2, act1, act2, lr):
    """
    activation: 'relu', 'sigmoid', 'linear', 'tanh', 'selu', 'elu'
    """
    model = Sequential()
    model.add(Dense(n1, input_dim=6, kernel_initializer='normal', activation=act1))
    model.add(Dense(n2, kernel_initializer='normal', activation=act2))
    model.add(Dense(1, kernel_initializer='normal'))
    
    opt = tensorflow.keras.optimizers.Adam(learning_rate=lr)
    
    model.compile(loss='mean_squared_error', optimizer=opt)
    return model

In [14]:
np.random.seed(seed)
# Create the NN framework
estimator = KerasRegressor(build_fn=neural_network_model, 
                           n1=60, n2=50, act1='relu', act2='linear', 
                           epochs=150, batch_size=100, verbose=0, lr=0.9)
# Fit to training data
history = estimator.fit(X_train, y_train, validation_split=0.30, epochs=150, 
        batch_size=10000, verbose=0)

print("Final MSE for train is %.3e and for validation is %.3e" % 
      (history.history['loss'][-1], history.history['val_loss'][-1]))

2022-03-09 14:19:50.388322: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-03-09 14:19:50.389678: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-09 14:19:50.394842: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2022-03-09 14:19:50.459912: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-03-09 14:19:50.461435: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3193885000 Hz


Final MSE for train is 4.365e+00 and for validation is 4.407e+00


In [22]:
def neural_network_model(n1, n2, act1, act2, lr):
    """
    activation: 'relu', 'sigmoid', 'linear', 'tanh', 'selu', 'elu'
    """
    model = Sequential()
    model.add(Dense(n1, input_dim=6, kernel_initializer='normal', activation=act1))
    model.add(Dense(n2, kernel_initializer='normal', activation=act2))
    model.add(Dense(1, kernel_initializer='normal'))
    
    opt = tensorflow.keras.optimizers.Adam(learning_rate=lr)
    
    model.compile(loss='mean_squared_error', optimizer=opt, metrics=['accuracy'])
    score, acc = model.evaluate(X_test, y_test)
    print('Test Score:', score)
    print('Test accuracy:', acc)
    return model

In [ ]:
np.random.seed(seed)

MSE = []
lr_vals = [0.00001, 0.0001, 0.01, 0.1, 1.0]
for lrate in lr_vals:
    estimator = KerasRegressor(build_fn=neural_network_model,
                               n1=60, n2=50, act1='relu', act2='linear', 
                               epochs=150, batch_size=10000, 
                               verbose=0, lr=lrate)
    history = estimator.fit(X_train, y_train, validation_split=0.30,
                            epochs=150, batch_size=10000, 
                            verbose=0)
    print("Final MSE for train is %.3e and for validation is %.3e" % 
      (history.history['loss'][-1], history.history['val_loss'][-1]))
    MSE.append([history.history['loss'][-1], 
                history.history['val_loss'][-1],
                estimator.model.evaluate(X_test, y_test)])

Final MSE for train is 5.896e+00 and for validation is 5.822e+00
103/103 [==============================] - 0s 3ms/step - loss: 5.8529
Final MSE for train is 4.965e+00 and for validation is 4.889e+00
103/103 [==============================] - 0s 2ms/step - loss: 4.9157
Final MSE for train is 1.369e-01 and for validation is 1.413e-01
103/103 [==============================] - 0s 3ms/step - loss: 0.1380


In [18]:
np.random.seed(seed)
# Create the NN framework
estimator = KerasRegressor(build_fn=neural_network_model, 
                           n1=100, n2=200, act1='sigmoid', act2='selu', 
                           epochs=150, batch_size=10000, verbose=0, lr=0.1)
# Fit to training data
history = estimator.fit(X_train, y_train, validation_split=0.30, epochs=150, 
        batch_size=10000, verbose=0)

print("Final MSE for train is %.3e and for validation is %.3e" % 
      (history.history['loss'][-1], history.history['val_loss'][-1]))

103/103 [==============================] - 0s 3ms/step - loss: 4.7809 - accuracy: 0.0000e+00
Test Score: 4.783041477203369
Test accuracy: 0.0
Final MSE for train is 1.410e-01 and for validation is 1.446e-01


In [19]:
activation_methods = ['relu', 'sigmoid', 'linear', 'tanh', 'selu', 'elu']

In [20]:
np.random.seed(seed)

for a1 in activation_methods:
    for a2 in activation_methods:
        estimator = KerasRegressor(build_fn=neural_network_model, 
                           n1=100, n2=200, act1=act1, act2=act2, 
                           epochs=150, batch_size=10000, verbose=0, lr=0.1)
        history = estimator.fit(X_train, y_train, validation_split=0.30,
                            epochs=150, batch_size=10000, 
                            verbose=0)
        print("Final MSE for train is %.3e and for validation is %.3e \n" % 
              (history.history['loss'][-1], history.history['val_loss'][-1]))

103/103 [==============================] - 4s 35ms/step - loss: 5.9605 - accuracy: 0.0000e+00
Test Score: 5.962210655212402
Test accuracy: 0.0
Final MSE for train is 1.381e-01 and for validation is 1.421e-01 

103/103 [==============================] - 3s 23ms/step - loss: 3.2946 - accuracy: 0.0000e+00
Test Score: 3.2949132919311523
Test accuracy: 0.0
Final MSE for train is 1.381e-01 and for validation is 1.422e-01 

103/103 [==============================] - 3s 26ms/step - loss: 6.0909 - accuracy: 0.0000e+00
Test Score: 6.089538097381592
Test accuracy: 0.0
Final MSE for train is 1.367e-01 and for validation is 1.417e-01 

103/103 [==============================] - 3s 25ms/step - loss: 5.8847 - accuracy: 0.0000e+00
Test Score: 5.886518478393555
Test accuracy: 0.0
Final MSE for train is 1.366e-01 and for validation is 1.415e-01 

103/103 [==============================] - 3s 23ms/step - loss: 5.9270 - accuracy: 0.0000e+00
Test Score: 5.92559814453125
Test accuracy: 0.0
Final MSE for tra

In [1]:
# hyperparameter optimization
import optuna
import sklearn.model_selection
import sklearn.preprocessing

In [16]:
def nn_val_loss(n1, n2, act1, act2, lr):
    estimator = KerasRegressor(build_fn=neural_network_model, 
                               n1=n1, n2=n2, act1=act1, act2=act2, 
                               epochs=150, batch_size=10000, verbose=0, lr=lr)
    history = estimator.fit(X_train, y_train, validation_split=0.30, epochs=150, 
                            batch_size=10000, verbose=0)
    
    return history.history['val_loss'][-1]

In [20]:
# define our objective that takes in a trial, picks some hyperparameters for that trial
# evaluates the hyperparameters
# then returns the value to optimize
def objective(trial):
    # define hyperparameter space
    act1 = trial.suggest_categorical('act1', ['relu', 'sigmoid', 'linear', 'tanh', 'selu', 'elu'])
    act2 = trial.suggest_categorical('act2', ['relu', 'sigmoid', 'linear', 'tanh', 'selu', 'elu'])
    n1 = trial.suggest_categorical('n1', [10, 50, 100, 150, 200,500])
    n2 = trial.suggest_categorical('n2', [10, 50, 100, 150, 200,500])
    lr = trial.suggest_categorical('lr', [0.00001, 0.0001, 0.001, 0.01, 0.1, 1.0])
    
    # get the score for the hyperparameters chosen
    score = nn_val_loss(n1, n2, act1, act2, lr)
    
    return score

# Create a study with the tep sampler
study = optuna.create_study(sampler=optuna.samplers.TPESampler(), direction='minimize')
# run the study.
# this uses TPE to try different hyperparameters (in this case x and y)
# and searches for the best ones
study.optimize(objective, n_trials=50)
# get the results as dataframe
results = study.trials_dataframe()

[I 2022-03-09 15:02:10,770] A new study created in memory with name: no-name-1b76d092-0f50-42ee-aa43-dfc22d2bae4f
[I 2022-03-09 15:02:41,311] Trial 0 finished with value: 0.14045622944831848 and parameters: {'act1': 'relu', 'act2': 'elu', 'n1': 100, 'n2': 100, 'lr': 0.1}. Best is trial 0 with value: 0.14045622944831848.
[I 2022-03-09 15:03:10,040] Trial 1 finished with value: 0.16008682548999786 and parameters: {'act1': 'linear', 'act2': 'relu', 'n1': 50, 'n2': 10, 'lr': 0.1}. Best is trial 0 with value: 0.14045622944831848.
[I 2022-03-09 15:03:41,434] Trial 2 finished with value: 0.985595703125 and parameters: {'act1': 'sigmoid', 'act2': 'elu', 'n1': 50, 'n2': 100, 'lr': 0.0001}. Best is trial 0 with value: 0.14045622944831848.
[I 2022-03-09 15:04:12,116] Trial 3 finished with value: 5.777660846710205 and parameters: {'act1': 'linear', 'act2': 'elu', 'n1': 200, 'n2': 200, 'lr': 1e-05}. Best is trial 0 with value: 0.14045622944831848.
[I 2022-03-09 15:04:43,260] Trial 4 finished with v

KeyboardInterrupt: 

In [ ]:
'act1': 'relu', 'act2': 'elu', 'n1': 100, 'n2': 100, 'lr': 0.1

In [23]:
np.random.seed(seed)
# Create the NN framework
estimator = KerasRegressor(build_fn=neural_network_model, 
                           n1=100, n2=100, act1='relu', act2='elu', 
                           epochs=150, batch_size=100, verbose=0, lr=0.1)
# Fit to training data
history = estimator.fit(X_train, y_train, validation_split=0.30, epochs=150, 
        batch_size=10000, verbose=0)

print("Final MSE for train is %.3e and for validation is %.3e" % 
      (history.history['loss'][-1], history.history['val_loss'][-1]))

103/103 [==============================] - 1s 9ms/step - loss: 5.8592 - accuracy: 0.0000e+00
Test Score: 5.860780715942383
Test accuracy: 0.0
Final MSE for train is 1.366e-01 and for validation is 1.411e-01


In [26]:
def nn_loss(n1, n2, act1, act2, lr):
    """
    activation: 'relu', 'sigmoid', 'linear', 'tanh', 'selu', 'elu'
    """
    model = Sequential()
    model.add(Dense(n1, input_dim=6, kernel_initializer='normal', activation=act1))
    model.add(Dense(n2, kernel_initializer='normal', activation=act2))
    model.add(Dense(1, kernel_initializer='normal'))
    
    opt = tensorflow.keras.optimizers.Adam(learning_rate=lr)
    
    model.compile(loss='mean_squared_error', optimizer=opt, metrics=['accuracy'])
    score, acc = model.evaluate(X_test, y_test)
    
    return score

In [28]:
def objective(trial):
    # define hyperparameter space
    act1 = trial.suggest_categorical('act1', ['relu', 'sigmoid', 'linear', 'tanh', 'selu', 'elu'])
    act2 = trial.suggest_categorical('act2', ['relu', 'sigmoid', 'linear', 'tanh', 'selu', 'elu'])
    n1 = trial.suggest_categorical('n1', [10, 50, 100, 150, 200,500])
    n2 = trial.suggest_categorical('n2', [10, 50, 100, 150, 200,500])
    lr = trial.suggest_categorical('lr', [0.00001, 0.0001, 0.001, 0.01, 0.1, 1.0])
    
    # get the score for the hyperparameters chosen
    score = nn_loss(n1, n2, act1, act2, lr)
    
    return score

# Create a study with the tep sampler
study = optuna.create_study(sampler=optuna.samplers.TPESampler(), direction='minimize')
# run the study.
# this uses TPE to try different hyperparameters (in this case x and y)
# and searches for the best ones
study.optimize(objective, n_trials=5000)
# get the results as dataframe
results = study.trials_dataframe()

[I 2022-03-09 15:37:50,428] A new study created in memory with name: no-name-f131f847-1285-4524-8fa2-a672c3c3ef32


103/103 [==============================] - 1s 9ms/step - loss: 5.9774 - accuracy: 0.0000e+00


[I 2022-03-09 15:37:51,600] Trial 0 finished with value: 5.978950023651123 and parameters: {'act1': 'sigmoid', 'act2': 'elu', 'n1': 10, 'n2': 10, 'lr': 0.001}. Best is trial 0 with value: 5.978950023651123.


103/103 [==============================] - 0s 3ms/step - loss: 6.4133 - accuracy: 0.0000e+00


[I 2022-03-09 15:37:52,190] Trial 1 finished with value: 6.415121078491211 and parameters: {'act1': 'sigmoid', 'act2': 'relu', 'n1': 100, 'n2': 50, 'lr': 0.1}. Best is trial 0 with value: 5.978950023651123.


...

[I 2022-03-09 17:25:24,953] Trial 4997 finished with value: 5.950137615203857 and parameters: {'act1': 'linear', 'act2': 'elu', 'n1': 50, 'n2': 500, 'lr': 0.0001}. Best is trial 3809 with value: 0.22065041959285736.


103/103 [==============================] - 2s 18ms/step - loss: 2.7054 - accuracy: 0.0000e+00


[I 2022-03-09 17:25:27,498] Trial 4998 finished with value: 2.7058002948760986 and parameters: {'act1': 'relu', 'act2': 'sigmoid', 'n1': 10, 'n2': 500, 'lr': 1e-05}. Best is trial 3809 with value: 0.22065041959285736.


103/103 [==============================] - 2s 20ms/step - loss: 3.5152 - accuracy: 0.0000e+00


[I 2022-03-09 17:25:30,028] Trial 4999 finished with value: 3.5134429931640625 and parameters: {'act1': 'tanh', 'act2': 'sigmoid', 'n1': 200, 'n2': 500, 'lr': 0.001}. Best is trial 3809 with value: 0.22065041959285736.


In [ ]:
'act1': 'selu', 'act2': 'sigmoid', 'n1': 100, 'n2': 500, 'lr': 1e-05

In [31]:
np.random.seed(seed)
# Create the NN framework
estimator = KerasRegressor(build_fn=neural_network_model, 
                           n1=100, n2=500, act1='selu', act2='sigmoid', 
                           epochs=150, batch_size=10000, verbose=0, lr=0.00001)
# Fit to training data
history = estimator.fit(X_train, y_train, validation_split=0.30, epochs=150, 
        batch_size=10000, verbose=0)

print("Final MSE for train is %.3e and for validation is %.3e" % 
      (history.history['loss'][-1], history.history['val_loss'][-1]))

print(estimator.score(X_test, y_test))

Final MSE for train is 2.014e+00 and for validation is 1.968e+00


In [34]:
print(estimator.score(X_test, y_test))

-1.98281729221344


In [46]:
def nn_val_loss(n1, n2, act1, act2, lr):
    estimator = KerasRegressor(build_fn=neural_network_model, 
                               n1=n1, n2=n2, act1=act1, act2=act2, 
                               epochs=150, batch_size=10000, verbose=0, lr=lr)
    estimator.fit(X_train, y_train, validation_split=0.30, epochs=150, 
                            batch_size=10000, verbose=0)
    y_predict = estimator.predict(X_test)
    r2 = r2_score(y_test, y_predict)
    
    return r2

In [47]:
def objective(trial):
    # define hyperparameter space
    act1 = trial.suggest_categorical('act1', ['relu', 'sigmoid', 'linear', 'tanh', 'selu', 'elu'])
    act2 = trial.suggest_categorical('act2', ['relu', 'sigmoid', 'linear', 'tanh', 'selu', 'elu'])
    n1 = trial.suggest_categorical('n1', [10, 50, 100, 150, 200,500])
    n2 = trial.suggest_categorical('n2', [10, 50, 100, 150, 200,500])
    lr = trial.suggest_categorical('lr', [0.00001, 0.0001, 0.001, 0.01, 0.1, 1.0])
    
    # get the score for the hyperparameters chosen
    score = nn_val_loss(n1, n2, act1, act2, lr)
    
    return score

# Create a study with the tep sampler
study = optuna.create_study(sampler=optuna.samplers.TPESampler(), direction='maximize')
# run the study.
# this uses TPE to try different hyperparameters (in this case x and y)
# and searches for the best ones
study.optimize(objective, n_trials=500)
# get the results as dataframe
results = study.trials_dataframe()

[I 2022-03-10 00:08:14,256] A new study created in memory with name: no-name-eb705a0d-cef3-4ef4-b708-a88f119395f6
[I 2022-03-10 00:08:47,261] Trial 0 finished with value: 0.0010995387605895468 and parameters: {'act1': 'selu', 'act2': 'elu', 'n1': 200, 'n2': 100, 'lr': 0.01}. Best is trial 0 with value: 0.0010995387605895468.
[I 2022-03-10 00:09:22,770] Trial 1 finished with value: -18.030495300086447 and parameters: {'act1': 'tanh', 'act2': 'tanh', 'n1': 500, 'n2': 50, 'lr': 0.0001}. Best is trial 0 with value: 0.0010995387605895468.
[I 2022-03-10 00:09:53,716] Trial 2 finished with value: -17.073840372736274 and parameters: {'act1': 'elu', 'act2': 'tanh', 'n1': 50, 'n2': 200, 'lr': 1.0}. Best is trial 0 with value: 0.0010995387605895468.


[I 2022-03-10 00:10:26,071] Trial 3 finished with value: -41.416096986135244 and parameters: {'act1': 'tanh', 'act2': 'elu', 'n1': 100, 'n2': 100, 'lr': 1e-05}. Best is trial 0 with value: 0.0010995387605895468.


[I 2022-03-10 00:10:59,994] Trial 4 finished with value: 0.004986284733510593 and parameters: {'act1': 'selu', 'act2': 'selu', 'n1': 200, 'n2': 200, 'lr': 0.01}. Best is trial 4 with value: 0.004986284733510593.


[I 2022-03-10 00:11:30,740] Trial 5 finished with value: -0.01432274998025207 and parameters: {'act1': 'tanh', 'act2': 'tanh', 'n1': 200, 'n2': 10, 'lr': 1.0}. Best is trial 4 with value: 0.004986284733510593.


[I 2022-03-10 00:12:04,394] Trial 6 finished with value: 0.004154558844385647 and parameters: {'act1': 'linear', 'act2': 'tanh', 'n1': 500, 'n2': 150, 'lr': 0.001}. Best is trial 4 with value: 0.004986284733510593.


...

[I 2022-03-10 05:01:51,293] Trial 495 finished with value: 0.0014637719478772793 and parameters: {'act1': 'relu', 'act2': 'linear', 'n1': 10, 'n2': 150, 'lr': 0.01}. Best is trial 287 with value: 0.015974041270554462.


[I 2022-03-10 05:02:26,383] Trial 496 finished with value: 0.013980320813360536 and parameters: {'act1': 'relu', 'act2': 'sigmoid', 'n1': 500, 'n2': 150, 'lr': 0.01}. Best is trial 287 with value: 0.015974041270554462.


[I 2022-03-10 05:03:03,808] Trial 497 finished with value: 0.007477483177238842 and parameters: {'act1': 'elu', 'act2': 'sigmoid', 'n1': 500, 'n2': 150, 'lr': 0.01}. Best is trial 287 with value: 0.015974041270554462.


[I 2022-03-10 05:03:37,217] Trial 498 finished with value: -0.8381312023390237 and parameters: {'act1': 'relu', 'act2': 'relu', 'n1': 500, 'n2': 150, 'lr': 0.1}. Best is trial 287 with value: 0.015974041270554462.


[I 2022-03-10 05:04:11,598] Trial 499 finished with value: 0.009063189757830603 and parameters: {'act1': 'tanh', 'act2': 'sigmoid', 'n1': 500, 'n2': 50, 'lr': 0.01}. Best is trial 287 with value: 0.015974041270554462.


###  'act1': 'elu', 'act2': 'relu', 'n1': 500, 'n2': 500, 'lr': 0.01

In [38]:
np.random.seed(seed)
# Create the NN framework
estimator = KerasRegressor(build_fn=neural_network_model, 
                           n1=500, n2=500, act1='elu', act2='relu', 
                           epochs=150, batch_size=10000, verbose=0, lr=0.01)
# Fit to training data
history = estimator.fit(X_train, y_train, validation_split=0.30, epochs=150, 
        batch_size=10000, verbose=0)

print("Final MSE for train is %.3e and for validation is %.3e" % 
      (history.history['loss'][-1], history.history['val_loss'][-1]))

Final MSE for train is 1.364e-01 and for validation is 1.396e-01


In [39]:
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score

In [40]:
y_predict = estimator.predict(X_test)
r2 = r2_score(y_test, y_predict)

In [41]:
r2

0.010671333144209094

In [42]:
mse = mean_squared_error(y_test, y_predict)

In [43]:
mse

0.13728992724605618

'act1': 'relu', 'act2': 'sigmoid', 'n1': 500, 'n2': 150, 'lr': 0.01

In [48]:
np.random.seed(seed)
# Create the NN framework
estimator = KerasRegressor(build_fn=neural_network_model, 
                           n1=500, n2=150, act1='relu', act2='sigmoid', 
                           epochs=150, batch_size=10000, verbose=0, lr=0.01)
# Fit to training data
history = estimator.fit(X_train, y_train, validation_split=0.30, epochs=150, 
        batch_size=10000, verbose=0)

print("Final MSE for train is %.3e and for validation is %.3e" % 
      (history.history['loss'][-1], history.history['val_loss'][-1]))

Final MSE for train is 1.352e-01 and for validation is 1.402e-01


In [49]:
y_predict = estimator.predict(X_test)